<a href="https://colab.research.google.com/github/solovee/Detector-de-Spam/blob/main/spamDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("venky73/spam-mails-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.86M/1.86M [00:00<00:00, 102MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/venky73/spam-mails-dataset/versions/1


In [12]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/venky73/spam-mails-dataset/versions/1/spam_ham_dataset.csv')

In [22]:
# Suponha que df_ tenha colunas: text_clean, label_num
df_0 = df[df["label_num"] == 0]
df_1 = df[df["label_num"] == 1]

# Pega o tamanho da menor classe
min_size = min(len(df_0), len(df_1))

# Balanceia cortando a maior
df_bal = pd.concat([
    df_0.sample(min_size, random_state=42),
    df_1.sample(min_size, random_state=42)
])

# Embaralha
df_bal = df_bal.sample(frac=1, random_state=42).reset_index(drop=True)


In [10]:
df.iloc[0]['text']

"Subject: enron methanol ; meter # : 988291\r\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

In [5]:
df['label'].value_counts()

,count
label,
ham,3672
spam,1499


In [24]:
def limpar_texto(texto):
    # 1. Remove "Subject:" (independente de maiúscula/minúscula)
    texto = re.sub(r"(?i)subject:\s*", "", texto)

    # 2. Remove quebras de linha e tabs
    texto = re.sub(r"[\r\n\t]+", " ", texto)

    # 3. Remove chaves e outros símbolos opcionais (se quiser mais limpo)
    texto = re.sub(r"[{}]+", "", texto)

    # 4. Troca múltiplos espaços por um só
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto

# Aplicar no DataFrame
df_bal["text_clean"] = df_bal["text"].apply(limpar_texto)

In [18]:
df['text_clean'].iloc[2]

"neon retreat ho ho ho , we ' re around to that most wonderful time of the year - - - neon leaders retreat time ! i know that this time of year is extremely hectic , and that it ' s tough to think about anything past the holidays , but life does go on past the week of december 25 through january 1 , and that ' s what i ' d like you to think about for a minute . on the calender that i handed out at the beginning of the fall semester , the retreat was scheduled for the weekend of january 5 - 6 . but because of a youth ministers conference that brad and dustin are connected with that week , we ' re going to change the date to the following weekend , january 12 - 13 . now comes the part you need to think about . i think we all agree that it ' s important for us to get together and have some time to recharge our batteries before we get to far into the spring semester , but it can be a lot of trouble and difficult for us to get away without kids , etc . so , brad came up with a potential alt

In [25]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

In [26]:
df_bal

,Unnamed: 0,label,text,label_num,text_clean
0,1940,ham,Subject: october flow at meter 982694 zavisch ...,0,october flow at meter 982694 zavisch # 2 i hav...
1,425,ham,Subject: fw : midcon 9401 ( permanent march fi...,0,fw : midcon 9401 ( permanent march file - expa...
2,1662,ham,Subject: eastrans nomination change effective ...,0,eastrans nomination change effective 9 / 23 / ...
3,5028,spam,Subject: a new era of online medical care .\r\...,1,a new era of online medical care . a new era o...
4,2804,ham,Subject: re : fw : first delivery - rodessa op...,0,re : fw : first delivery - rodessa operating c...
...,...,...,...,...,...
2993,5045,spam,Subject: nova psp ! disponivel ja so na spring...,1,nova psp ! disponivel ja so na springshop info...
2994,1352,ham,Subject: re : formosa meter # : 1000\r\nthe de...,0,re : formosa meter # : 1000 the deal will cont...
2995,3424,ham,Subject: august is known for hot deals !\r\nde...,0,august is known for hot deals ! dear golfboxx ...
2996,963,ham,"Subject: enron actuals for june 12 , 2000\r\nt...",0,"enron actuals for june 12 , 2000 teco tap 117 ..."


In [27]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_bal["text_clean"].tolist(), df_bal["label_num"].tolist(), test_size=0.2, random_state=42
)

# Tokenizer BERT base uncased
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenizar
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
val_encodings   = tokenizer(val_texts, truncation=True, padding=True, max_length=64)

# Criar Dataset customizado
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextDataset(train_encodings, train_labels)
val_dataset   = TextDataset(val_encodings, val_labels)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [31]:
from transformers import BertForSequenceClassification

# Carregar BERT pré-treinado com cabeça de classificação
num_labels = len(set(df["label_num"]))  # número de classes
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)  # multi-classe
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),        # use 'binary' se for binário
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted")
    }


In [34]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,

    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Treinar
trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: leo-solovijovas (leo-solovijovas-instituto-federal-de-educa-o) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.643800
20,0.488800
30,0.407200
40,0.322500
50,0.208000
60,0.182000
70,0.159400
80,0.118500
90,0.150000
100,0.100100


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=900, training_loss=0.06860552853025081, metrics={'train_runtime': 5070.2056, 'train_samples_per_second': 1.419, 'train_steps_per_second': 0.178, 'total_flos': 236602616532480.0, 'train_loss': 0.06860552853025081, 'epoch': 3.0})

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",  # avalia a cada época
    save_strategy="epoch"         # salva checkpoint a cada época
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [35]:
metrics = trainer.evaluate()
print(metrics)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.12271329015493393, 'eval_runtime': 128.2381, 'eval_samples_per_second': 4.679, 'eval_steps_per_second': 0.585, 'epoch': 3.0}


In [54]:
import torch

# Exemplo de string nova
novo_texto = "return me later"

# Tokenizar
inputs = tokenizer(
    novo_texto,
    truncation=True,
    padding=True,
    max_length=64,  # mesmo valor usado no treino
    return_tensors="pt"  # retorna tensores PyTorch
)


In [55]:
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
!pip install --upgrade transformers


In [56]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits


In [57]:
logits

tensor([[-3.5193,  3.5167]])

In [58]:
pred = torch.argmax(logits, dim=-1).item()
print("Classe prevista:", pred)


Classe prevista: 1


In [ ]:
# Pasta onde será salvo
save_path = "./bert_finetuned_model"

# Salvar modelo
model.save_pretrained(save_path)

# Salvar tokenizer
tokenizer.save_pretrained(save_path)


In [60]:
import shutil

# Caminho da pasta que você quer zipar
path = "./bert_finetuned_model"

# Nome do arquivo zip que será criado (sem a extensão)
zip_name = "./bert_finetuned_model_zip"

# Criar o zip
shutil.make_archive(zip_name, 'zip', path)

print(f"Pasta {path} zipada como {zip_name}.zip")


Pasta ./bert_finetuned_model zipada como ./bert_finetuned_model_zip.zip
